In [1]:
%load_ext autoreload
%autoreload 2

In [2]:
def decompose_to_elementary_spans (layer,text):
    spanindexes = []
    attributes = layer.attributes
    
    for s in layer.spans:
        spanstart = s.start
        spanend = s.end
        spanindexes.append(spanstart)
        spanindexes.append(spanend)
    
    html_spans=[]
    
    spanindexes = sorted(set(spanindexes))
    
    for i in range(len(spanindexes)-1):
        span_text = text[spanindexes[i]:spanindexes[i+1]]
        html_spans.append([spanindexes[i],spanindexes[i+1],span_text])
                    
    for i, html_span in enumerate(html_spans):
        span_list=[]
        span_start = html_span[0]
        span_end = html_span[1]
        for s in layer.spans:
            if span_start in range (s.start,s.end):
                span_list.append(s)
        html_span.append(span_list)
        del html_span[0:2]

    return html_spans

In [3]:
def decorate(html_spans):
    spans = []
    
    for i in range(len(html_spans)):
        coordinates = []
        coordinates.append(html_spans[i][0])
        coordinates.append(html_spans[i][1])
        spans.append(coordinates)
        
    #contents_without_attributes = fun_output[1]
    #spans_in_html_spans = fun_output[2]
    #texts = fun_output[3]
    #attributes = getattr(text,layer).attributes
    #contents = []
    
    #for content in contents_without_attributes:
    #    new_content=[]
    #    for i in range(len(content)):
    #        new_content.append(attributes[i%len(attributes)])
    #        new_content.append(content[i])
    #    contents.append(new_content)
    
    #tabledata=[]
    
    #for content in contents_without_attributes:
    #    for i in range(len(attributes)):
    #        tabledata.append(attributes[i])
    #        tabledata.append(content[i])
    
    texts = []
    
    for span in html_spans:
        content=[]
        span_info = span[2]
        for i in range(len(span_info)):
            content.append(span_info[i].text)
        texts.append(content)
    
    fragments = []
    last_end = 0
    print(texts)
    style='background-color:blue;'
    
    for i, (s, e) in enumerate(spans):
        fragments.append(text[last_end:s])
        fragments.append('<span class="span')
        if len(html_spans[i][2])>1:
            fragments.append(" overlapping-span")
        fragments.append('"; onclick="visualise{0}(')
        fragments.append(str(html_spans[i][1]).replace('"','').replace('{','{{').replace('}','}}'))
        fragments.append(',')
        fragments.append(str(i))
        fragments.append(',')
        fragments.append("["+str(html_spans[i][0])+","+str(html_spans[i][1])+"]")
        fragments.append(", this,")
        fragments.append(str(texts[i]))
        fragments.append(')"; indexes="')
        fragments.append("["+str(html_spans[i][0])+","+str(html_spans[i][1])+"]")
        fragments.append('"; span_exists = "";')
        fragments.append('style = "')
        fragments.append(style)
        fragments.append('">')
        fragments.append(text[s:e])
        fragments.append('</span>')
        last_end = e
    fragments.append(text[last_end:])
    return ''.join(fragments)

In [4]:
def is_pure_text(segment):
    for span in segment:
        if len(segment[1])>0:
            return False
    return True

In [5]:
class SpanDecorator:
    # call
    # css
    pass

In [6]:
class DirectPlainSpanVisualiser(SpanDecorator):
    
    default_conf_colour = "red"
    
    def __init__(self, colour_mapping=None, bg_mapping=None, fill_empty_spans=False):
        self.bg_mapping = bg_mapping or self.default_bg_mapping
        self.colour_mapping = colour_mapping
        self.fill_empty_spans = fill_empty_spans
        #self.colour_mapping = colour_mapping or default_color_mapping
        pass
    
    def __call__(self, segment):
        
        output=''
    
        for span in segment:
            # Simple text no span to fill
            if not self.fill_empty_spans and self.is_pure_text(span):
                output+=span[0]
            else:
                # There is a span to decorate
                output += '<span style='
                if self.colour_mapping is not None:
                    output += 'color:' + self.color_mapping(span)
                if self.bg_mapping is not None:
                    output += 'background:'+ self.bg_mapping(span)
                output += '>'

                output += span[0]
                output += '</span>'

        return output

    def default_bg_mapping(self, segment):
        if len(segment[1]) > 1:
            return self.default_conf_colour
    
        return "yellow"
    
    def is_pure_text(self,segment):
        if len(segment[1])>0:
            return False
        return True

In [7]:
def test_bg_mapping(segment):
    if 'SADA' in segment[1][0].attr_1:
        return "green"
    else:
        return "yellow"

In [8]:
from estnltk.tests import new_text

In [9]:
new_text(5).layer_1[0].attr_1

['SADA']

In [10]:
def bg_mapper_discrete(segment, attribute, value_mapping, default_color, conflict_color):
    
    if segment[1].length() > 1:
        return conflict_color
    
    return value_mapping.get(segment[1][0].get_attr(attribute), default_color)    

In [11]:
#from estnltk.FancySpanVisualisation import DisplayAttributes as Display
from IPython.display import display_html

segments = decompose_to_elementary_spans(new_text(5).layer_1,new_text(5).text)

#span_decorator = DirectPlainSpanVisualiser(bg_mapping=lambda segment: bg_mapper_discrete(segment, concrete value))
span_decorator = DirectPlainSpanVisualiser(bg_mapping=test_bg_mapping)

t = span_decorator(segments)

print(t)
#display = Display(decorator=span_decorator)

display_html(t, raw=True)

<span style=background:green>Sada</span> <span style=background:yellow>kaks</span><span style=background:yellow>kümme</span><span style=background:yellow>nd</span> <span style=background:yellow>kolm</span>.<span style=background:yellow> Neli</span> <span style=background:yellow>tuhat</span> <span style=background:yellow>viis</span><span style=background:green>sada</span> <span style=background:yellow>kuus</span><span style=background:yellow>kümme</span><span style=background:yellow>nd</span> <span style=background:yellow>seitse</span> <span style=background:yellow>koma</span> <span style=background:yellow>kaheksa</span>. <span style=background:yellow>Üheksa</span><span style=background:yellow>kümme</span><span style=background:yellow>nd</span>


Sada kaks kümme nd kolm . Neli tuhat viis sada kuus kümme nd seitse koma kaheksa . Üheksa kümme nd

In [12]:
elementary_spans = decompose_to_elementary_spans(new_text(5).layer_1,new_text(5).text)
elementary_spans

[['Sada', [AS[Annotation(Sada, {'attr': 'L1-0', 'attr_1': 'SADA'})]]],
 [' ', []],
 ['kaks',
  [AS[Annotation(kaks, {'attr': 'L1-1', 'attr_1': 'KAKS'})],
   AS[Annotation(kakskümmend, {'attr': 'L1-2', 'attr_1': 'KAKS'}), Annotation(kakskümmend, {'attr': 'L1-2', 'attr_1': 'KÜMME'}), Annotation(kakskümmend, {'attr': 'L1-2', 'attr_1': 'KAKSKÜMMEND'})]]],
 ['kümme',
  [AS[Annotation(kakskümmend, {'attr': 'L1-2', 'attr_1': 'KAKS'}), Annotation(kakskümmend, {'attr': 'L1-2', 'attr_1': 'KÜMME'}), Annotation(kakskümmend, {'attr': 'L1-2', 'attr_1': 'KAKSKÜMMEND'})],
   AS[Annotation(kümme, {'attr': 'L1-3', 'attr_1': 'KÜMME'})]]],
 ['nd',
  [AS[Annotation(kakskümmend, {'attr': 'L1-2', 'attr_1': 'KAKS'}), Annotation(kakskümmend, {'attr': 'L1-2', 'attr_1': 'KÜMME'}), Annotation(kakskümmend, {'attr': 'L1-2', 'attr_1': 'KAKSKÜMMEND'})]]],
 [' ', []],
 ['kolm', [AS[Annotation(kolm, {'attr': 'L1-4', 'attr_1': 'KOLM'})]]],
 ['.', []],
 [' Neli', [AS[Annotation( Neli, {'attr': 'L1-5', 'attr_1': 'NELI'})]

In [13]:
from estnltk import Text

In [14]:
def test_morph_bg_mapping(segment):
    if 'V' in segment[1][0].partofspeech:
        return "green"
    else:
        return "yellow"

In [15]:
tekst = Text('''Nagu Kataloonias ikka, saab otsustavaks reedene päev, mil kihutatakse kruusal.\n
Hispaania väga siledal ja kiirel asfaldil on esimesel päeval saadud kaotust üsna keeruline tagasi sõita. \n
„Poole minutiga allajäämine reede õhtuks on palju,” selgitas Tänak. „Pluss-miinus kümme sekundit on okei.”''')
tekst.tag_layer(['morph_analysis'])
tekst.morph_analysis[0].partofspeech

segments = decompose_to_elementary_spans(tekst.morph_analysis,tekst.text)

#span_decorator = DirectPlainSpanVisualiser(bg_mapping=lambda segment: bg_mapper_discrete(segment, concrete value))
span_decorator = DirectPlainSpanVisualiser(bg_mapping=test_morph_bg_mapping)

t = span_decorator(segments)

#display = Display(decorator=span_decorator)

display_html(t, raw=True)

Nagu Kataloonias ikka , saab otsustavaks reedene päev , mil kihutatakse kruusal . 

 Hispaania väga siledal ja kiirel asfaldil on esimesel päeval saadud kaotust üsna keeruline tagasi sõita . 

 „ Poole minutiga allajäämine reede õhtuks on palju ,” selgitas Tänak . „ Pluss-miinus kümme sekundit on okei .”